# Fáza 2 - Predspracovanie údajov

__Autori:__ Dávid Penťa, Samuel Bernát
__Percentuálny podiel práce:__ 50% / 50%

V tejto fáze sa od Vás očakáva že realizujte predspracovanie údajov pre strojové učenie. Výsledkom bude upravená dátová sada (csv alebo tsv), kde jedno pozorovanie je opísané jedným riadkom.
- scikit-learn vie len numerické dáta, takže treba niečo spraviť s nenumerickými dátami.
- Replikovateľnosť predspracovania na trénovacej a testovacej množine dát, aby ste mohli
zopakovať predspracovanie viackrát podľa Vašej potreby (iteratívne).

Keď sa predspracovaním mohol zmeniť tvar a charakteristiky dát, je možné že treba realizovať EDA opakovane podľa Vašej potreby. Bodovanie znovu (EDA) nebudeme, zmeny ale dokumentujte. Problém s dátami môžete riešiť iteratívne v každej fáze aj vo všetkých fázach podľa potreby.

In [128]:
import statsmodels.api as sm
import dateparser as dateparser
import matplotlib
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as py
import statsmodels.stats.api as sms
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import pearsonr

In [129]:
measurements_file = "data/measurements.csv"
measurements_data = pd.read_csv(measurements_file, sep='\t')

stations_file = "data/stations.csv"
stations_data = pd.read_csv(stations_file, sep='\t')

## Integrácia a čistenie dát
Transformujte dáta na vhodný formát pre strojové učenie t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte.
- Pri riešení chýbajúcich hodnôt (missing values) vyskúšajte rôzne stratégie ako napr.
    - odstránenie pozorovaní s chýbajúcimi údajmi
    - nahradenie chýbajúcej hodnoty mediánom, priemerom, pomerom (ku korelovanému atribútu), alebo pomocou lineárnej regresie resp. kNN
- Podobne postupujte aj pri riešení vychýlených hodnôt (outlier detection):
    - odstránenie vychýlených (odľahlých) pozorovaní
    - nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (5% resp. 95%)

In [130]:
measurements_data.dropna(inplace=True)
stations_data.dropna(inplace=True)

stations_data["QoS"] = np.where(stations_data["QoS"] == "accep", "acceptable", stations_data["QoS"])
stations_data["QoS"] = np.where(stations_data["QoS"] == "maitennce", "maintenance", stations_data["QoS"])

# stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).strftime('%B-%d-%Y'))
# stations_data['revision_timestamp'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())
stations_data['revision'] = stations_data['revision'].apply(lambda x: pd.Timestamp(x).timestamp())

stations_data['latitude'] = stations_data['latitude'].round(5)
stations_data['longitude'] = stations_data['longitude'].round(5)

stations_data["station"] = np.where(stations_data["station"] == "T‚Äôaebaek", "Taebaek", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "'Ali Sabieh", "Ali Sabieh", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Oktyabr‚Äôskiy", "Oktyabrsk", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Roslavl‚Äô", "Roslavl", stations_data["station"])
stations_data["station"] = np.where(stations_data["station"] == "Dyat‚Äôkovo", "Dyatkovo", stations_data["station"])

stations_data

,QoS,station,code,latitude,longitude,revision
0,good,Casa Blanca,MX,19.04222,-98.11889,1.574813e+09
1,building,Mikhaylovka,RU,50.06000,43.23790,1.501027e+09
2,building,Shahre Jadide Andisheh,IR,35.68030,51.01930,1.370045e+09
3,building,Aracaju,BR,-10.91111,-37.07167,1.356739e+09
4,maintenance,Parola,IN,20.88098,75.11937,1.562285e+09
...,...,...,...,...,...,...
1105,building,Tadmur,SY,34.56240,38.28402,1.373414e+09
1106,average,Jizzax,UZ,40.11583,67.84222,1.369872e+09
1107,acceptable,West Chester,US,39.96097,-75.60804,1.351987e+09
1108,acceptable,Oktyabrsk,RU,54.48147,53.47103,1.429315e+09


In [131]:
stations_data = stations_data.groupby(by='station').apply(lambda x: x.loc[x['revision']==x['revision'].max()])

In [132]:
df=pd.merge(stations_data, measurements_data, on=['latitude', 'longitude'], how='right')

df

,QoS,station,code,latitude,longitude,revision,PM10,CO,Pb,C2H3NO5,...,O3,TEMP,NOx,SO2,NH3,CH4,PRES,PM2.5,warning,PAHs
0,maintenance,Roslavl,RU,53.95278,32.86389,1.505174e+09,6.33846,6.61796,44.89247,0.22646,...,6.93304,18.26344,7.24953,8.60015,5.35805,6.00130,1136.95323,10.59506,1.0,9.24391
1,acceptable,Lesnoy,RU,58.63667,59.80222,1.444694e+09,6.78253,7.32578,50.42226,0.27588,...,6.41183,13.61953,8.25103,8.93315,5.16975,5.07507,1148.76311,10.04912,1.0,8.73640
2,maintenance,Robertsonpet,IN,12.95629,78.27539,1.382573e+09,5.71508,9.26158,26.86094,0.18788,...,7.94233,16.76223,4.10062,4.44370,5.76106,7.01203,1089.36057,10.53430,1.0,8.67231
3,good,Dublin,US,32.54044,-82.90375,1.620864e+09,9.16375,7.12481,48.82457,1.32986,...,11.63958,12.48813,6.26932,7.81736,5.96896,8.44913,1099.39295,9.40919,0.0,4.80834
4,acceptable,Riverbank,US,37.73604,-120.93549,1.614211e+09,7.04390,7.96828,56.56011,0.31311,...,10.42120,12.74538,8.75889,7.43447,8.58263,6.89506,1050.67737,7.20484,0.0,6.00215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11349,excellent,Recife,BR,-8.05389,-34.88111,1.654733e+09,9.56528,8.54654,40.70338,1.89160,...,7.55099,25.45745,7.64682,8.18163,5.81816,8.36536,1107.16252,9.30018,0.0,3.88821
11350,acceptable,Gorgonzola,IT,45.53069,9.40531,1.548202e+09,9.19695,7.92486,59.23293,1.40122,...,10.79559,13.34113,6.78988,8.45646,9.75596,6.93416,1149.48262,9.42584,1.0,9.22787
11351,average,Glendale,US,34.14251,-118.25508,1.511827e+09,10.27894,6.99879,48.01565,3.83467,...,9.42860,5.17211,8.98672,7.14161,8.29618,4.34404,1106.63191,11.02506,1.0,11.22798
11352,average,Azrou,MA,33.43443,-5.22126,1.420157e+09,8.21318,7.03138,50.31096,0.54791,...,9.37725,0.68008,7.38683,7.91926,9.26976,6.81885,1095.25573,8.53529,1.0,8.19965


In [133]:
# df.drop(['revision_timestamp'], axis=1)

In [134]:
df['QoS_num'] = -1

df.loc[df.QoS == "excellent", "QoS_num"] = "1"
df.loc[df.QoS == "good", "QoS_num"] = "2"
df.loc[df.QoS == "average", "QoS_num"] = "3"
df.loc[df.QoS == "acceptable", "QoS_num"] = "4"
df.loc[df.QoS == "building", "QoS_num"] = "5"
df.loc[df.QoS == "maintenance", "QoS_num"] = "5"

df[['QoS_ID']] = df[['QoS_num']].apply(pd.to_numeric)

In [138]:
df['station_ID'] = df['station'].rank(method='dense')
df['code_ID'] = df['code'].rank(method='dense')

In [139]:
df

,QoS,station,code,latitude,longitude,revision,PM10,CO,Pb,C2H3NO5,...,NH3,CH4,PRES,PM2.5,warning,PAHs,QoS_num,QoS_ID,station_ID,code_ID
0,maintenance,Roslavl,RU,53.95278,32.86389,1.505174e+09,6.33846,6.61796,44.89247,0.22646,...,5.35805,6.00130,1136.95323,10.59506,1.0,9.24391,5,5,487.0,91.0
1,acceptable,Lesnoy,RU,58.63667,59.80222,1.444694e+09,6.78253,7.32578,50.42226,0.27588,...,5.16975,5.07507,1148.76311,10.04912,1.0,8.73640,4,4,320.0,91.0
2,maintenance,Robertsonpet,IN,12.95629,78.27539,1.382573e+09,5.71508,9.26158,26.86094,0.18788,...,5.76106,7.01203,1089.36057,10.53430,1.0,8.67231,5,5,483.0,51.0
3,good,Dublin,US,32.54044,-82.90375,1.620864e+09,9.16375,7.12481,48.82457,1.32986,...,5.96896,8.44913,1099.39295,9.40919,0.0,4.80834,2,2,156.0,110.0
4,acceptable,Riverbank,US,37.73604,-120.93549,1.614211e+09,7.04390,7.96828,56.56011,0.31311,...,8.58263,6.89506,1050.67737,7.20484,0.0,6.00215,4,4,481.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11349,excellent,Recife,BR,-8.05389,-34.88111,1.654733e+09,9.56528,8.54654,40.70338,1.89160,...,5.81816,8.36536,1107.16252,9.30018,0.0,3.88821,1,1,474.0,13.0
11350,acceptable,Gorgonzola,IT,45.53069,9.40531,1.548202e+09,9.19695,7.92486,59.23293,1.40122,...,9.75596,6.93416,1149.48262,9.42584,1.0,9.22787,4,4,209.0,53.0
11351,average,Glendale,US,34.14251,-118.25508,1.511827e+09,10.27894,6.99879,48.01565,3.83467,...,8.29618,4.34404,1106.63191,11.02506,1.0,11.22798,3,3,205.0,110.0
11352,average,Azrou,MA,33.43443,-5.22126,1.420157e+09,8.21318,7.03138,50.31096,0.54791,...,9.26976,6.81885,1095.25573,8.53529,1.0,8.19965,3,3,38.0,63.0
